In [1]:
import io
import os
import sys

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.optimizers import Adamax
from sklearn.metrics import classification_report

import innvestigate
import innvestigate.utils as iutils

from Model_NoDef import DFNet
from plot_result import plot_history
from data_loader import load_npz

2025-06-30 23:15:30.962983: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-30 23:15:31.643586: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-06-30 23:15:31.643662: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2025-06-30 23:15:31.643669: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found. Using CPU.')
else:
  print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
DATASET_DIR = "/home/kwakrhkr59/XAI_WF/preprocessing/output/data.npz"
feature = "size"
NB_EPOCH = 5
BATCH_SIZE = 128
VERBOSE = 1
LENGTH = 5000
OPTIMIZER = tf.keras.optimizers.legacy.Adamax(learning_rate=0.002, beta_1=0.9, beta_2=0.999)
INPUT_SHAPE = (LENGTH, 1)

In [4]:
print("Loading and preparing data for training, and evaluating the model")
X_train, y_train, X_valid, y_valid, X_test, y_test = load_npz(DATASET_DIR)
NB_CLASSES = y_train.shape[1]

Loading and preparing data for training, and evaluating the model


Configured NB_CLASSES (75) does not match actual number of classes (95). Using 95.


In [5]:
X_train.shape

(15200, 5000, 2)

In [6]:
X_train = X_train[:, :, 0].reshape(X_train.shape[0], X_train.shape[1], 1)
X_valid = X_valid[:, :, 0].reshape(X_valid.shape[0], X_valid.shape[1], 1)
X_test = X_test[:, :, 0].reshape(X_test.shape[0], X_test.shape[1], 1)

print("Shape of X_train:", X_train.shape)
print("Shape of X_valid:", X_valid.shape)
print("Shape of X_test:", X_test.shape)

Shape of X_train: (15200, 5000, 1)
Shape of X_valid: (1900, 5000, 1)
Shape of X_test: (1900, 5000, 1)


In [7]:
print("Building and training DF model")
model = DFNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)

model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER, metrics=["accuracy"])
print("Model compiled")

Building and training DF model
Model compiled


In [8]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [9]:
history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=NB_EPOCH,
    verbose=VERBOSE,
    validation_data=(X_valid, y_valid)
)

Epoch 1/5


2025-06-30 23:15:40.075483: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2025-06-30 23:15:40.079098: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2025-06-30 23:15:40.079155: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:237] Used ptxas at ptxas
2025-06-30 23:15:40.079462: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


119/119 [==============================] - 12s 58ms/step - loss: 4.7869 - accuracy: 0.0297 - val_loss: 4.4967 - val_accuracy: 0.0221
Epoch 2/5
119/119 [==============================] - 5s 44ms/step - loss: 4.3800 - accuracy: 0.0590 - val_loss: 4.2816 - val_accuracy: 0.0642
Epoch 3/5
119/119 [==============================] - 5s 44ms/step - loss: 4.1493 - accuracy: 0.0797 - val_loss: 3.8564 - val_accuracy: 0.1263
Epoch 4/5
119/119 [==============================] - 5s 44ms/step - loss: 3.9571 - accuracy: 0.1028 - val_loss: 3.9720 - val_accuracy: 0.1174
Epoch 5/5
119/119 [==============================] - 5s 44ms/step - loss: 3.7731 - accuracy: 0.1299 - val_loss: 3.6716 - val_accuracy: 0.1558


In [10]:
score_test = model.evaluate(X_test, y_test, verbose=VERBOSE)
y_pred = model.predict(X_test)

print(f"Testing accuracy: {score_test[1]:.4f}")
print(f"Testing loss: {score_test[0]:.4f}")

60/60 [==============================] - 1s 4ms/step
Testing accuracy: 0.1537
Testing loss: 3.6909


In [11]:
plot_history(history, score_test)

<Figure size 640x480 with 0 Axes>

In [ ]:
def apply_lrp_1d(model, data_array, pred_index=None, method="lrp.z"):
    model_wo_softmax = iutils.model_wo_softmax(model)
    analyzer = innvestigate.create_analyzer(method, model_wo_softmax, neuron_selection_mode="index")
    preds = model.predict(data_array)
    if pred_index is None:
        pred_index = np.argmax(preds[0])
    analysis = analyzer.analyze(data_array, neuron_selection=pred_index)
    lrp_relevance = np.squeeze(np.sum(np.abs(analysis[0]), axis=-1))
    max_relevance = lrp_relevance.max()
    if max_relevance > 0:
        lrp_relevance /= max_relevance
    return lrp_relevance

def plot_lrp_relevance(original_data, lrp_relevance, title="", output_path=None, relevance_threshold=0.7):
    plt.figure(figsize=(15, 8))
    plt.subplot(2, 1, 1)
    plt.plot(original_data, color='blue', label='Original Data', linewidth=1.5)
    highlight_indices = np.where(lrp_relevance > relevance_threshold)[0]
    if len(highlight_indices) > 0:
        breaks = np.where(np.diff(highlight_indices) != 1)[0] + 1
        segments = np.split(highlight_indices, breaks)
        first_segment = True
        for segment in segments:
            if len(segment) > 0:
                start_idx = segment[0]
                end_idx = segment[-1]
                plt.axvspan(start_idx, end_idx, color='red', alpha=0.3,
                            label=f'High Relevance (>{relevance_threshold})' if first_segment else "")
                first_segment = False
    plt.title(f"{title}\nOriginal Data with LRP Highlights (Threshold: {relevance_threshold})", fontsize=14)
    plt.xlabel("Sequence Index", fontsize=12)
    plt.ylabel("Value", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    plt.subplot(2, 1, 2)
    plt.bar(range(len(lrp_relevance)), lrp_relevance, width=1.0, color='darkorange', alpha=0.8, label='LRP Relevance Score')
    plt.axhline(y=relevance_threshold, color='green', linestyle='--', label='Relevance Threshold', alpha=0.7)
    plt.title(f"LRP Relevance Score", fontsize=14)
    plt.xlabel("Sequence Index", fontsize=12)
    plt.ylabel("Relevance Score (0-1)", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    if output_path:
        plt.savefig(output_path, dpi=300)
        plt.close()
    else:
        plt.show()

lrp_output_dir = f"results_lrp/{feature}_batch{BATCH_SIZE}_epoch{NB_EPOCH}_per_class/"
os.makedirs(lrp_output_dir, exist_ok=True)

desired_samples_per_class = 5

print("\nGenerating LRP visualizations for test samples...")

class_names = [f"Class_{i}" for i in range(NB_CLASSES)]

for class_idx in range(NB_CLASSES):
    current_class_name = class_names[class_idx]
    print(f"\nProcessing LRP for class: {current_class_name}")
    class_indices = np.where(y_test[:, class_idx] == 1)[0]
    num_samples_to_plot = min(desired_samples_per_class, len(class_indices))
    if num_samples_to_plot == 0:
        print(f"No samples found for class: {current_class_name} in test set. Skipping LRP.")
        continue
    np.random.shuffle(class_indices)
    for sample_idx_in_class in range(num_samples_to_plot):
        i = class_indices[sample_idx_in_class]
        data_sample_for_lrp = np.expand_dims(X_test[i], axis=0)
        true_label_idx = np.argmax(y_test[i])
        preds = model.predict(data_sample_for_lrp)
        predicted_label_idx = np.argmax(preds[0])
        lrp_relevance_map = apply_lrp_1d(model, data_sample_for_lrp,
                                         pred_index=predicted_label_idx,
                                         method="lrp.z")
        sample_title = (f"{current_class_name} Sample {sample_idx_in_class+1}\n"
                        f"True: {class_names[true_label_idx]} / Pred: {class_names[predicted_label_idx]}")
        plot_lrp_relevance(np.squeeze(X_test[i]), lrp_relevance_map,
                           title=sample_title,
                           output_path=os.path.join(lrp_output_dir, f"LRP_Class_{class_idx}_Sample_{i}.png"))

print("LRP visualizations generation completed.")


Generating LRP visualizations for test samples...

Processing LRP for class: Class_0
1/1 [==============================] - 0s 155ms/step


AttributeError: module 'innvestigate.utils' has no attribute 'model_wo_softmax'

: 